# Introduction to C/C++

## Logistics

Before we start, make sure you can compile and run a C program.
On Linux or Mac, open a terminal; on Windows, open a Ubuntu Windows
Subsystem for Linux (WSL) terminal.
Then type:
```bash
gcc --version
g++ --version
```
If you see version information, your compiler is ready.
Otherwise, you may need to install your compiler tool chain.
On Linux and WSL, type
```bash
sudo apt update
sudo apt install gcc g++
```
After installation, you should be able to see version information of
`gcc` and `g++`.